In [1]:
from rl4co.envs.scheduling.jssp.env import JSSPEnv

generator_params = {
"num_jobs" : 3 ,
"num_machines": 5 ,
"min_processing_time": 1 ,
"max_processing_time": 99
}

env = JSSPEnv(generator_params = generator_params)


C:\Users\soner\anaconda3\envs\env_rl4co\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
td = env.reset(batch_size=[3])

------this is action mask---------------
tensor([[[ True,  True,  True, False,  True],
         [ True, False,  True,  True,  True],
         [False,  True,  True,  True,  True]],

        [[False,  True,  True,  True,  True],
         [False,  True,  True,  True,  True],
         [ True,  True,  True,  True, False]],

        [[ True,  True,  True,  True, False],
         [False,  True,  True,  True,  True],
         [ True,  True,  True, False,  True]]])
------this is action mask---------------


In [3]:
# get_action_mask is the problem


In [4]:
import torch
if torch.cuda.is_available():
    accelerator = "gpu"
    batch_size = 4
    train_data_size = 2_000
    embed_dim = 128
    num_encoder_layers = 4
else:
    accelerator = "cpu"
    batch_size = 4
    train_data_size = 1_000
    embed_dim = 64
    num_encoder_layers = 2

In [5]:
from rl4co.models import L2DPolicy4PPO

# Policy: neural network, in this case with encoder-decoder architecture
policy = L2DPolicy4PPO(
    embed_dim=embed_dim,
    num_encoder_layers=num_encoder_layers,
    env_name="jssp",
    het_emb=False
)

In [6]:
import torch


def make_step(td, decoder):
    """
    Equivalent to FJSP make_step(), adapted for JSSP where no encoder is used.
    td: TensorDict representing the current state of the environment.
    decoder: The L2DDecoder or policy that generates action logits.
    env: The JSSP environment instance.
    """
    # Directly decode logits and mask from the raw input state `td`
    hidden, _ = decoder.feature_extractor(td)

    logits, mask = decoder(td, num_starts=0 , hidden = hidden)

    # Mask invalid actions by setting their logits to -inf
    action = logits.masked_fill(~mask, -torch.inf).argmax(1)

    # Update the state with the selected action
    td["action"] = action

    # Step the environment with the selected action
    td = env.step(td)["next"]

    return td

In [19]:
# import time
# from matplotlib import pyplot as plt
# from IPython.core.display_functions import clear_output
#
# env.render(td, 0)
# # Update plot within a for loop
# while not td["done"].all():
#     # Clear the previous output for the next iteration
#     #clear_output(wait=True)
#
#     td = make_step(td=td ,decoder = policy.decoder)
#     env.render(td, 0)
#     # Display updated plot
#     display(plt.gcf())
#
#     # Pause for a moment to see the changes
#     time.sleep(.4)

In [10]:
from rl4co.envs.scheduling.djssp.env import DJSSPEnv

generator_params = {
"num_jobs" : 3 ,
"num_machines": 5 ,
"min_processing_time": 1 ,
"max_processing_time": 99 ,
"mtbf": 17 ,
    "mttr":2
}
djssp_env = DJSSPEnv(generator_params=  generator_params)

In [15]:
td = djssp_env.reset(batch_size = [2])

In [16]:
td["time"]

tensor([52.5877, 32.7373])

In [18]:
td["job_arrival_times"]

tensor([[52.5877, 55.3098, 66.6287],
        [32.7373, 44.9308, 46.9802]])

In [20]:
td["action_mask"]

tensor([[False,  True, False, False],
        [False,  True, False, False]])

In [22]:
djssp_env._get_job_machine_availability(td)[0]

tensor([[ True,  True, False,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True]])